# Obstacle avoidance
The obstacle avoidance is directly inside the Thymio robot. It uses the proximity sensors to detect obstacles and avoid them.\
If the robot detects an obstacle it will turn left or right depending on which side the obstacle is.\
If the robot detects an obstacle is state will change to 1.\
This state is synchronized with the state of the robot in the navigation thread so when it leave this avoidance state it will ask for a new global path.\
The speed of each wheel of the robot are controlled by the navigation thread and synchronized with tdmclient.

## Avoidance code inside the Thymio
```python
speed_left_global = 0
speed_right_global = 0
obstSpeedGain = [10, 4, -2, -6, -12]
state_global = 0
DELTA = 1
PROX_TRESHOLD = 200
timer_period[0] = 10  # 10ms sampling time

@onevent 
def timer0():
    global prox_ground_delta, prox_horizontal, speed0, speedGain,obstSpeedGain, motor_left_target, motor_right_target
    global speed_left_global, speed_right_global, state_global, motor_left_speed, motor_right_speed
    spLeft = speed_left_global
    spRight = speed_right_global
    state_global = 0
    side_obstacle = False
    # adjustment for obstacles ("gradient" due to obstacles)
    for i in range(5):
        spLeft += prox_horizontal[i] * obstSpeedGain[i] // 100
        spRight += prox_horizontal[i] * obstSpeedGain[4 - i] // 100
        if prox_horizontal[i] > PROX_TRESHOLD:
            state_global = 1
        if 0 < i and i < 4:
            if (prox_horizontal[0] > PROX_TRESHOLD or prox_horizontal[4] > PROX_TRESHOLD ) and prox_horizontal[i] < PROX_TRESHOLD:
                side_obstacle = True
            else:
                side_obstacle = False

    if side_obstacle:
        if prox_horizontal[0] > PROX_TRESHOLD:
            spLeft += 5
            spRight += 15
        if  prox_horizontal[4] > PROX_TRESHOLD:
            spLeft += 15
            spRight += 5  


    # motor control
    motor_left_target = spLeft
    motor_right_target = spRight



## Syncronization function with the navigation thread
```python
@tdmclient.notebook.sync_var
def thymio_speed(speed_left,speed_right):
    global speed_left_global
    global speed_right_global
    speed_right_global = speed_right
    speed_left_global = speed_left

@tdmclient.notebook.sync_var
def thymio_state(has_global_path):
    global state_global
    thymio_state = "Local"
    if state_global == 1:
        thymio_state = "Avoidance"
        has_global_path = False
    return thymio_state, has_global_path

def thymio_turn(angle_diff):
    if abs(angle_diff) > 10:
        if angle_diff > 0:
            thymio_speed(-50,50)
            time.sleep(0.05*angle_diff)
            thymio_speed(0,0)
            time.sleep(0.1)
        else:
            thymio_speed(50,-50)
            time.sleep(0.05*abs(angle_diff))
            thymio_speed(0,0)
            time.sleep(0.1)
        return angle_diff*math.pi/180 # return in radians
    return 0
